# Plotting demos
Demos the plotting functions and some related utility

In [1]:
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt

In [2]:
import os
os.chdir('../')

In [3]:
from roc_bike_growth.loader import POI_graph_from_polygon, bike_infra_from_polygon, carall_from_polygon
from roc_bike_growth.plotting import add_categorical_legend

In [4]:
rochester = ox.geocode_to_gdf('rochester, ny').geometry[0]

## Plot bike infra network by type

In [5]:
bike_infra = bike_infra_from_polygon(rochester, compose_all=False)

No OSM data for bike_designatedpath
No OSM data for bike_cyclewayrighttrack
No OSM data for bike_cyclewaylefttrack
No OSM data for bike_cyclestreet
No OSM data for bike_bicycleroad
No OSM data for bike_livingstreet


### Interactive map with colors

In [86]:
colors = {
    'bike_cyclewaylane': '#0e79ca', #  this == "bike lanes" bike lanes
    'bike_cyclewaytrack': '#c469d8', # cycle track
    'bike_highwaycycleway': '#28cf08', # bike blvd -- this == "trails"
    'bike_designatedpath': '#fcbc2d', # trail == "shared use / bike blvd"
}

In [101]:
m = folium.Map(location=[43.15, -77.6750], tiles="Stamen Toner", zoom_start=13)
for name, g in bike_infra:
    layer = folium.FeatureGroup(name=name,show=True)
    ox.folium.plot_graph_folium(
    g,
    layer, 
    popup_attribute='name', 
     **{'color': colors[name]})
    layer.add_to(m)
    

In [102]:
add_categorical_legend(m, 'Infra type', colors.values(), colors.keys())

### Adding POIs as markers

In [89]:
POIs = POI_graph_from_polygon(rochester)

In [90]:
pois_gdf = ox.utils_graph.graph_to_gdfs(POIs, nodes=True, edges=False)

In [103]:
poi_layer = folium.FeatureGroup(name='pois',show=True)
for id, row in pois_gdf.iterrows():
    poi_layer.add_child(folium.CircleMarker(
        location=(row['y'],row['x']),
        tooltip=row['name'],
        radius=6,
        **{'color':'#e07a5f', 'fill':True}
    ))
poi_layer.add_to(m)

In [104]:
folium.LayerControl(position='bottomright').add_to(m)

In [105]:
m

In [106]:
m.save('map.html')